In [1]:
from bs4 import BeautifulSoup
import re
import os
import urllib.request as urllib2
import pandas as pd

%run aerofoils.ipynb

naca1.dat failed. ERROR: float division by zero


In [2]:
def get_AFT_cases(directory='dat/case-dat'):
    baseFlpth = "http://airfoiltools.com"

    html_all = urllib2.urlopen("http://airfoiltools.com/search/airfoils").read()
    soup_all = BeautifulSoup(html_all,'html.parser')    
    
    links_all = [link['href'] for link in soup_all.find_all('a', href=re.compile("/airfoil/details"))]
    
    linknames = [link[25:-3].lower() for link in links_all]
    filenames_dir = [str(file)[11:].lower() for file in os.scandir(directory)]
    linknames_new = []
    for name in linknames:
        count = 0
        for filename in filenames_dir:
            if name in filename:
                count += 1
            else:
                continue
        if count > 3:
            continue
        else:
            linknames_new.append(name)
    links_new = []
    for new in linknames_new:
        for link in links_all:
            if new in link:
                links_new.append(link)
                    
    print('Staring AFT Case download...')
    indx = 0
    for link in links_new:
        html_foil = urllib2.urlopen(baseFlpth + link).read()
        soup_foil = BeautifulSoup(html_foil,'html.parser')
        
        links_Re = [link_Re['href'] for link_Re in soup_foil.find_all('a', href=re.compile("/polar/details"))]
        for link_Re in links_Re:
            if link_Re.endswith('-n5'):
                continue
            elif link_Re.endswith('-n1'):
                continue

            html_Re = urllib2.urlopen(baseFlpth + link_Re).read()
            soup_Re = BeautifulSoup(html_Re,'html.parser')
            
            link_csv = soup_Re.find_all('a', href=re.compile("polar/csv"))[0]['href']
            
            name = link_csv[20:] + '.csv'
            fullfilename = os.path.join(directory, name)
            urllib2.urlretrieve(baseFlpth+link_csv, fullfilename)
        
            indx += 1
        
    print(f' Done. {indx} files copied from http://airfoiltools.com/search/airfoils and saved to ~/{directory}.')


def get_RENNES_cases(directory='dat/rennes-dat/case-dat'):
    baseFlpth = "https://perso.univ-rennes1.fr/laurent.blanchard/Profils/"

    html_all = urllib2.urlopen(baseFlpth).read()
    soup_all = BeautifulSoup(html_all,'html.parser') 
        
    links_all = [link['href'] for link in soup_all.find_all('a', href=re.compile("/index"))]
    links_all.remove('centrepoussee/index.html')
    links_all.remove('clouet/index.html')
    
    linknames = [link[:-11].lower() for link in links_all]
    filenames_dir = [str(file)[11:].split('-')[0].lower() for file in os.scandir(directory)]
    linknames_new = []
    for name in linknames:
        count = 0
        for filename in filenames_dir:
            if name in filename:
                count += 1
            else:
                continue
        if count > 5:
            continue
        else:
            linknames_new.append(name)
    links_new = []
    for new in linknames_new:
        for link in links_all:
            if new in link:
                links_new.append(link)
                                    
    print('Staring RENNES Case download...')
    indx = 0
    for link in links_new: 
        try:
            html_foil = urllib2.urlopen(baseFlpth + link).read()
            soup_foil = BeautifulSoup(html_foil,'html.parser')

            links_Re = [link_Re['href'] for link_Re in soup_foil.find_all('a', href=re.compile(".txt"))]

            for link_Re in links_Re:
                name = link[:-11] + '-' + link_Re
                fullfilename = os.path.join(directory, name)
                urllib2.urlretrieve(baseFlpth+link[:-10]+link_Re, fullfilename)

                indx += 1
            
        except Exception as e:
            pass  # print(e)
    
    print(f' Done. {indx} files copied from {baseFlpth} and saved to ~/{directory}.')

In [3]:
# get_AFT_cases(directory='dat/case-dat')
# get_RENNES_cases(directory='dat/rennes-dat/case-dat')

In [4]:
def read_AFTcase(file):
    top = pd.read_csv(file, nrows=8)
    bottom = pd.read_csv(file, skiprows=9)
    
    name = top.iloc[1,0][:-3].lower().replace('.','').replace('_','-')
    Re = float(top.iloc[2,0])
    
    alphas = []
    Cls = []
    Cds = []
    for indx, row in bottom.iterrows():
        alpha = float(row.Alpha)
        Cl = float(row.Cl)
        Cd = float(row.Cd)
        
        alphas.append(alpha)
        Cls.append(Cl)
        Cds.append(Cd)
    
    return name, Re, alphas, Cls, Cds


def read_RENcase(file):
    with open(file, "r") as f:
        lines = f.readlines()
        dat = []
        for line in lines:
            line = line.strip()
            dat.append(line)
    
    name = str(file)[11:-6].split('-')[0].replace('_','-')
    Re = float(dat[8][28:33]) * 10**6
    dat = dat[12:]
    
    alphas = []
    Cls = []
    Cds = []
    for row in dat:
        row = row.split('  ')
        alpha = float(row[0])
        Cl = float(row[1])
        Cd = float(row[2])

        alphas.append(alpha)
        Cls.append(Cl)
        Cds.append(Cd)
        
    return name, Re, alphas, Cls, Cds


def read_EXPcase(file):
    dat = pd.read_csv(file)
    
    name = dat.file.tolist()
    Re = dat.Re.tolist()
    alphas = dat.alpha.tolist()
    Cls = dat.Cl.tolist()
    Cds = dat.Cd.tolist()
    
    return name, Re, alphas, Cls, Cds


def create_cases(directory='dat/case-dat', ext='csv'):
    cases_df = pd.DataFrame(columns=['file', 'Re', 'alpha', 'Cl', 'Cd'])
    
    for file in os.scandir(directory):
        if file.name.endswith('.' + ext):
            try:
                if directory == 'dat/case-dat':
                    name, Re, alphas, Cls, Cds = read_AFTcase(file)
                    name = [name.lower()] * len(alphas)
                    Re = [Re] * len(alphas)
                elif directory == 'dat/rennes-dat/case-dat':
                    name, Re, alphas, Cls, Cds = read_RENcase(file)
                    name = [name.lower()] * len(alphas)
                    Re = [Re] * len(alphas)
                elif directory == 'dat/exp-dat/case-dat':
                    name, Re, alphas, Cls, Cds = read_EXPcase(file)
                    
                case_df = pd.DataFrame({'file': name, 'Re': Re, 'alpha': alphas, 'Cl': Cls, 'Cd': Cds})
                cases_df = pd.concat([cases_df, case_df], ignore_index=True)

            except Exception as e:
                pass  # print(e)
    
    cases_df = cases_df.sort_values(['file', 'Re', 'alpha'], ignore_index=True)
    return cases_df


def merge(cases_df, aerofoils_df):
    cases_df['right_index'] = cases_df.index.tolist()

    totdata_df = pd.merge(aerofoils_df, cases_df, on='file', how='outer', indicator=True)
    data_df = totdata_df.loc[totdata_df._merge == 'both'].drop(columns=['_merge'])

    dup, inds = list(data_df.right_index.duplicated()), data_df.index.tolist()
    dup_inds = [i for i, d in zip(inds,dup) if d is True]
    data_df = data_df[~data_df.index.isin(dup_inds)].drop(columns=['right_index'])
    data_df = data_df.sort_values(['file', 'Re'], ascending=[True, True])

    return cases_df, totdata_df, data_df


def save_cases(df, file):
    df.to_csv(file, index=False)
    
    
def df_from_csv(file):
    df = pd.read_csv(file)
    return df

In [5]:
# cases_df = create_cases(directory='dat/case-dat', ext='csv')
# save_cases(df=cases_df, file='dat-saved/cases-df.csv')
# 
# ren_cases_df = create_cases(directory='dat/rennes-dat/case-dat', ext='txt')
# save_cases(df=ren_cases_df, file='dat-saved/ren-cases-df.csv')
# 
# exp_cases_df = create_cases(directory='dat/exp-dat/case-dat', ext='csv')
# save_cases(df=exp_cases_df, file='dat-saved/exp-cases-df.csv')

In [6]:
cases_df = df_from_csv(file='dat-saved/cases-df.csv')
ren_cases_df = df_from_csv(file='dat-saved/ren-cases-df.csv')
exp_cases_df = df_from_csv(file='dat-saved/exp-cases-df.csv')

In [7]:
cases_df, totdata_df, data_df = merge(aerofoils_df=aerofoils_df, cases_df=cases_df)
print('Normal:', len(cases_df), len(totdata_df), len(data_df))

ren_cases_df, ren_totdata_df, ren_data_df = merge(aerofoils_df=ren_aerofoils_df, cases_df=ren_cases_df)
print('Rennes:', len(ren_cases_df), len(ren_totdata_df), len(ren_data_df))

exp_cases_df, exp_totdata_df, exp_data_df = merge(aerofoils_df=aerofoils_df, cases_df=exp_cases_df)
print('Experimental:', len(exp_cases_df), len(exp_totdata_df), len(exp_data_df))

Normal: 829544 831325 829544
Rennes: 31526 31527 27641
Experimental: 1012 2664 1012


In [8]:
# data_df.sample(frac=0.00001).drop(columns = ['spline', 'xy_profile'])
# list(set(ren_data_df.file.tolist()))